In [187]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import os
from sklearn.metrics import mean_squared_error

In [188]:
channels = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7'
            , 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz'
            , 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2','AUX_1', 'ACC_X', 'ACC_Y', 'ACC_Z']
removed_channels = ['Fp1', 'F8', 'F7', 'Fp2', 'F3', 'F4']
# TODO ADD AUX WHEN CONNECTED
eye_data_names = ['x', 'y', 'z', 'left_blink', 'right_blink', 'both_blink', 'left_openness', 'right_openness']

columns =  eye_data_names + channels

data, header = pyxdf.load_xdf('data/LSLDataV2/LSLDataLeftFourTimes2/eeg-and-eyetracking.xdf')


In [189]:
def get_time_stamps(stamps: list):
    start = []
    end = []
    more_than_one = False

    for i in range(len(stamps)):
        if i == (len(stamps)-1):
            if(stamps[i-1] == (stamps[i] - 1)):
                end.append(stamps[i])
            break

        if(stamps[i] == (stamps[i+1] - 1)):
            if not more_than_one:
                start.append(stamps[i])
            more_than_one = True
            continue
        else:
            if more_than_one:
                end.append(stamps[i])
                more_than_one = False

    return start, end

In [190]:
def plot_eeg_blink_single_norm(df,df_ts, start, end,column,rang = 50, scale=1, flip = False):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    axis.plot(df_ts['time'][start-rang:end+rang], (df_norm + plus) *(-1 if flip else 1) *scale, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Amplitude ")
    axis.set_title(column)
    plt.legend()
    plt.show()

def plot_eeg_blink_single(df,df_ts, start, end,column,rang = 50):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")

    axis.plot(df_ts['time'][start-rang:end+rang], df[column], label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], df['left_openness'], color='orange', label = "Eye openness left - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], df['right_openness'], color='violet', label = "Eye openness right - SRanipal")
    axis.set_title(column)
    plt.legend()
    plt.show()

In [191]:

def init_data(data, shift = 0):
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = columns
        df_stream['AUX_1'] = df_stream['AUX_1']+ 1000000
        if shift != 0:
            df_stream['AUX_1'] = df_stream['AUX_1'].shift(shift)
        return stream, df_stream, df_stream_ts

stream, df_stream, df_stream_ts = init_data(data)
def get_blinks_in_df(df):
    left_blinks = df.index[(df['left_blink'] >= 1) & (df['right_blink'] != 1)].tolist()
    right_blinks = df.index[(df['right_blink'] >= 1) & (df['left_blink'] != 1)].tolist()
    both_blink = df.index[df['both_blink'] == 1].tolist()
    return left_blinks,right_blinks, both_blink

left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)

In [192]:
avg_time = np.diff(df_stream_ts.to_numpy().flatten()).tolist()

In [193]:
start, end = get_time_stamps(left_blinks)
flip = False
chosen_channel = 'AUX_1'
print(start)
print(end)


[1201, 1510, 1858]
[1278, 1576, 1917]


In [194]:
from scipy.signal import butter, lfilter
# Sample rate and desired cutoff frequencies (in Hz).
fs = 500
lowcut = 1
highcut = 28



def butter_bandpass(lowcut, highcut, fs, order=4):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def plot_eeg_blink_single_norm_butter(df,df_ts, start, end,column,rang = 50, scale=1, flip = False, order = 4):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]

    y = butter_bandpass_filter((df_norm + plus) *(-1 if flip else 1) *scale, lowcut, highcut, fs, order=order)
    peaks, _ = signal.find_peaks(y[rang-7:rang+30])
    peaks = [peak + rang-7 for peak in peaks]

    axis.plot(df_ts['time'][start-rang:end+rang], y, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Normalized Amplitude")
    for i in peaks:
        plt.plot(df_ts['time'][i+ start - rang], y[i], marker="o", markersize=5, markeredgecolor="turquoise", markerfacecolor="black", label="Local maximum " + str(y[i].round(2)) + ", " + str(i+ start - rang))
    axis.set_title(column)
    plt.legend()
    plt.show()

def get_peaks(df, start, end, column, rang=50, scale=1, order=4, flip=False):
    df = df[start - rang:end + rang]
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    y = butter_bandpass_filter((df_norm + plus) *(-1 if flip else 1) *scale, lowcut, highcut, fs, order=order)
    peaks, _ = signal.find_peaks(y, height=0)
    return peaks[np.argmax([y[i] for i in peaks])]+ start - rang ## Get only highest blink onset



In [195]:


def find_shift(start, peaks):
    differences_start = []
    differences_end = []
    for i in range(len(peaks)):
        differences_start.append(start[i]-peaks[i])
    return differences_start

def plot_table(df):
    #make this example reproducible

    #define figure and axes
    fig, ax = plt.subplots()
    #hide the axes
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')

    #create data

    #create table
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    table.scale(2.5, 2.5)
    table.set_fontsize(30)
    plt.show()

In [196]:

def run_files(paths, path='data/LSLDataV2/', both = True, plot = False, shift_init=0):
    peak_info = []
    avg_time = []
    for (index,y) in enumerate(paths):
        data, header = pyxdf.load_xdf(path+str(y)+'/eeg-and-eyetracking.xdf')
        stream, df_stream, df_stream_ts = init_data(data,shift_init)
        left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)
        start, end = get_time_stamps(both_blink if both else left_blinks)
        avg_time.append(np.diff(df_stream_ts.to_numpy().flatten()).tolist())
        peaks = []
        for i in range(len(start)):
            peaks.append(get_peaks(df_stream, start[i], end[i], chosen_channel, rang=50, scale=5, flip=both, order=4))

        shift = find_shift(start, peaks)
        peak_info.append(shift)

    n = len(max(peak_info, key=len))
    t = [x + [None]*(n-len(x)) for x in peak_info]
    numpy_array_peak_info = np.array(t)
    df_peak = pd.DataFrame(numpy_array_peak_info)
    df_peak = df_peak.set_index([paths])
    df_peak['time between index (s)'] = [i[0] for i in avg_time]
    df_peak['Sum (indexes)'] = df_peak.sum(axis=1)
    df_peak['Avg (indexes)'] = round(df_peak.mean(axis=1),3)
    df_peak['Avg time (s)'] = round(df_peak['Avg (indexes)']*df_peak['time between index (s)'],5)
    if plot:
        plot_table(df_peak)
    return df_peak



In [197]:
paths = [name for name in os.listdir("./data/LSLDataV2")]
both = [name for name in paths if "Both" in name]
left = [name for name in paths if "Left" in name]


df_both = run_files(both, both=True , plot=False)
display(df_both)
df_left = run_files(left, both=False , plot=False)
display(df_left)

,0,1,2,3,4,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLData4BothBlink,-13,-10,None,None,None,0.008333,-22.991667,-11.496,-0.09580
LSLDataBothFiveTimesFIR,-7,-10,-10,-20,None,0.008333,-46.991667,-15.664,-0.13053
LSLDataBothSlow4Times2,-8,-11,-10,14,-13,0.008333,-27.991667,-7.998,-0.06665
LSLDataBothSlow4TimesFIR,-11,-6,-10,-9,None,0.008333,-35.991667,-11.997,-0.09998


,0,1,2,3,4,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeft5times2FIR,-28,-26,-21,-22,-24,0.008333,-120.991667,-34.569,-0.28808
LSLDataLeft5TimesFIR,-19,-21,-23,-23,-24,0.008333,-109.991667,-31.426,-0.26188
LSLDataLeftFourTimes2,-17,-19,-20,None,None,0.008333,-55.991667,-22.397,-0.18664


In [198]:
def get_offset(df):
    avg_offset = df['Avg time (s)'].sum()/df.shape[0]
    avg_offset_index = df['Avg (indexes)'].sum()/df.shape[0]
    #print(avg_offset_index)
    return avg_offset, avg_offset_index


In [200]:
print("(Seconds, Number of indexes to shift) ---- Both")
print(get_offset(df_both))
print("(Seconds, Number of indexes to shift) ---- Left")
print(get_offset(df_left))

df_both_shift = run_files(both, both=True , plot=False, shift_init=round(get_offset(df_both)[1]))
df_both_shift.loc['Average']= df_both_shift.sum(numeric_only=True, axis=0)/df_both_shift.shape[0]
display(df_both_shift)
df_left_shift = run_files(left, both=False , plot=False, shift_init=round(get_offset(df_left)[1]))
df_left_shift.loc['Average']= df_left_shift.sum(numeric_only=True, axis=0)/df_left_shift.shape[0]
display(df_left_shift)

print(get_offset(df_both_shift))
print(get_offset(df_left_shift))

(Seconds, Number of indexes to shift) ---- Both
(-0.09824000000000001, -11.78875)
(Seconds, Number of indexes to shift) ---- Left
(-0.24553333333333335, -29.464)


,0,1,2,3,4,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLData4BothBlink,-1,2,None,None,None,0.008333,1.008333,0.50400,0.004200
LSLDataBothFiveTimesFIR,5,2,2,-8,None,0.008333,1.008333,0.33600,0.002800
LSLDataBothSlow4Times2,4,0,2,25,-1,0.008333,30.008333,8.57400,0.071450
LSLDataBothSlow4TimesFIR,1,6,2,3,None,0.008333,12.008333,4.00300,0.033360
Average,NaN,NaN,NaN,NaN,NaN,0.008333,11.008333,3.35425,0.027952


,0,1,2,3,4,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeft5times2FIR,1,3,8,8,5,0.008333,25.008333,7.145000,0.059540
LSLDataLeft5TimesFIR,10,9,6,6,5,0.008333,36.008333,10.288000,0.085730
LSLDataLeftFourTimes2,12,10,9,None,None,0.008333,31.008333,12.403000,0.103360
Average,NaN,NaN,NaN,NaN,NaN,0.008333,30.675000,9.945333,0.082877


(0.027952499999999998, 3.3542499999999995)
(0.08287666666666667, 9.945333333333332)
